In [1]:
%gui qt5
# need to wait for gui qt5 to completely load before running next cell
import numpy as np
from neurom.io import swc
import napari

/Users/victor/github/napari/napari/__init__.py:27: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


Read swc downloaded from http://www.neuromorpho.org/neuron_info.jsp?neuron_name=204-2-6nj

data_block is np.array-like with the following columns:
    
    [X, Y, Z, R, TYPE, ID, P]
    X(float): x-coordinate
    Y(float): y-coordinate
    Z(float): z-coordinate
    R(float): radius
    TYPE(integer): one of the types described by POINT_TYPE
    ID(integer): unique integer given to each point, the `ROOT_ID` is -1
    P(integer): the ID of the parent

In [2]:
data = swc.read('204-2-6nj.CNG.swc').data_block

In [3]:
parents = data[:,6]

# function to get edge set from parent column in swc file
def get_edges(parents):
    """Returns (len(parents) - 1) by 2 array. Each row is an edge
    """
    edges = np.zeros((len(parents)-1,2),dtype = int)
    for idx, parent in enumerate(parents):
        if parent >= 0:
            edges[idx-1,0] = int(parent - 1)
            edges[idx-1,1] = idx
    
    return edges

edges = get_edges(parents)
edges

array([[  0,   1],
       [  0,   2],
       [  0,   3],
       ...,
       [599, 600],
       [600, 601],
       [601, 602]])

In [4]:
# function to convert data into list of paths to be passed into napari
def parse_swc(data):
    
    # compute location of branches
    break_points = [0] + list(np.nonzero(np.diff(data[:, 6]) < 0)[0]+1) + [len(data)-1]
    paths = []
    
    # use location of branches to separate branches into individual paths
    for i in range(len(break_points)-1):
        if break_points[i+1] - break_points[i] > 2:
            paths.append(data[break_points[i]:break_points[i+1], :3])
    return paths

In [5]:
paths = parse_swc(data)
viewer = napari.Viewer(ndisplay=3)
viewer.add_shapes(paths, shape_type='path', edge_color='blue', name='swc')

<Shapes layer 'swc' at 0x12eee2dd0>

![Screenshot from napari. napari does not have ability to retreive snapshot from 3D render](napari_neurons.png)

Edge widths from data can vary from each edge. napari cannot scale edge widths

In [ ]:
# radii from swc file
swc_radii = data[:,3]

# radii from napari paths
napari_radii = viewer.layers['swc'].edge_width

print("Edge widths from swc (first 5)")
print(swc_radii[0:5],'\n')
print("Edge width from napari path (one radius for all edges)")
print(napari_radii)

In [12]:
paths[1]

array([[-11.11,  59.05,   3.03],
       [-14.21,  61.81,   1.16],
       [-14.67,  62.55,  -0.69],
       [-18.35,  64.33,  -2.55],
       [-20.21,  66.93,  -2.8 ],
       [-20.34,  68.45,  -3.51],
       [-21.35,  69.18,  -6.29],
       [-22.73,  70.76,  -6.61],
       [-27.07,  71.38,  -8.84],
       [-31.55,  70.26, -12.17]])